# Notes

This notebook was originally created and stored on Google Colaboratory. For convenience, output has been saved according to the intended input (stored on Google Drive). This is for display and not intended to be ran locally.

# Simulating user-side recommendation
This notebook will simulate what will happen on the user's side: a user will 'download' the U and V matrix (built from clustering and preforming WNMF) and from a subset of the user's own reviews, find the most similar cluster. Once the most similar cluster is found the remaining reviews will be tested for error against the cluster's predicted value.

In order to run the simulation, this module will also handle test user selection. 

In [0]:
# Mounting drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# DONE: read U and V files
# DONE: from Jonathon's user dict, read it and find the top users.
# DONE: from the top users, find their reviews and make a dictionary <User, {set of users' businesses they reviewed}>
# DONE: from each users review set, compare how many businesses are in the V matrix
# DONE: for each user with at least 3 businesses in the V matrix, preform cross-validation for each user (separate it as evenly as possibly, find which cluster it belongs to for the subset, test the rest of them for prediction, repeat)

## Reading the U and V matrix
We import pandas and numpy, then assign the main filepaths here. Finally we use pandas to read the files as dataframes.

In [0]:
import pandas as pd
import numpy as np
import scipy.stats

filepath_u = '/content/gdrive/My Drive/Summer Research 2019/Individual Work Folder/Minh/u.csv'
filepath_v = '/content/gdrive/My Drive/Summer Research 2019/Individual Work Folder/Minh/v.csv'

df_u = pd.read_csv(filepath_u, index_col=0)
df_v = pd.read_csv(filepath_v, index_col=0)

df_v.head(10)

,"""A-Ri-Rang"": ZMcbVIEXsLO7j1Q1GXKPSw","""Activities and Recreation Center - ARC"": zAD08AT1GgX-cVNy9iMq4A","""Aldi"": tDTUSKxPUUkpFaxBZZl4Cw","""Alexander's Steakhouse"": CpNMXASiwtJv5eCDf0n63g","""Alexandra's"": rOOsFYRPiTOcOnaiChJklQ","""All Creatures Animal Hospital"": zkieW2F82--bbOIRhOCj0g","""Alto Vineyards"": M8G8S2takaE_NOVutGzEkg","""Am-Ko Oriental Foods & Gifts"": zPPw_vUatLmvnRpW1OoX6w","""Amtrak"": jZeLcFDCfBhAxjq1Owe65A","""Anderson Dental"": N-IwlZQimaUrxYF_jGUCOg","""Animal Emergency Clinic of Champaign County"": BhfQ8Rh4L-ca_so-3QjWoA","""Anita Purves Nature Center"": vV6oNhupyM8gEhjmhqLPjQ","""Antonio's Pizza"": BoTQepQTjGbTXA4g4Hj-WA","""Applebee's Neighborhood Grill & Bar"": r_QcMIgY2zxdcV40kK_rxw","""Armored Gopher Games"": cn999G2xZNMj8ZYDot0C8w","""Aroma Cafe"": 0FrYsoVHheQGoXEQsH2d2Q","""Art Coop"": gZI007iLBe_LZbrHSDxHtg","""Art Mart"": dVBp-ayxdvwG95BYfqKwDw","""Assembly Hall"": V7CbZXLAG_wg-wZfQxCZcA","""Atlanta Bread"": nG59soBP2SaQ25rf2n-0bA","""B Won"": C_Zj7H7bUJ5kGCwFmEwW6w","""BJ Grand Salon & Spa"": ChjKd6HgLKmUAQnsrpDSBA","""Bacaro"": jeTfL2kCyBtmFGSrSQHqVw","""Bacca Cigar"": u68_uFbvsqrwZVeHo-a71g","""Bagelmen's Inc"": xo_53Ec0MacJEC5ux7dWhQ","""Bar Guiliani"": tEkUEOsxj9h0pzClYrGXMQ","""Barfly"": 67STv2NXNvt7K8sdbjqetQ","""Basil Thai Cafe"": XtJj67rKT16a4tQw7bxtyw","""Basil Thai Urbana"": w5Y_FiGPSlPemJVxAZZToQ","""Basmati Indian Cuisine"": 7U7uLS9YjjhBM1SoyNH30w","""Bella Mia Boutique"": 2t8Z3VwvV6YpW2gnP-VHig","""Bentley's Pub"": u4D2VypIx3nMsRRbOwx-sA","""Bevande Coffee"": N5PfEojrY4rFqpqzno4aZg","""Biaggi's Ristorante Italiano"": qeJnMI5RmyJ2TZvkqvjn6Q","""Big Lots - Champaign"": OYpRlyukZZh9zH2mxxv4rg","""Big Mouth's"": Z2W3K8x9cRGXvlGWerZH-w","""Billy Barooz Bar & Grill"": PmVxbit6HDDsEUS-j9aDfg","""Blain's Farm & Fleet"": tLgKerW16F9V239MjNA2PA","""Blues"": Q2bnRzJ8AC-3lWyQY8DZqA","""Bo Bo China"": EVkytEhlC1nswqmgrHdviw",...,"""The Bread Company"": Ah4i15g8Ow_zphzcpulTxQ","""The Cake Artist's Studio"": rNbAIbTn0zN8yNk0IV7dfQ","""The Canopy Club"": SVXpyYPAuvJVKcfZ0nMKyg","""The Estate Sale"": pWxCiLYvlUHd9TVJ7n0mEQ","""The Fubar Lounge"": PWs6xJQJPHxknd3FcdKn9Q","""The Great Impasta"": pH72Y8aqqJlq3bgtj-d3UA","""The Habitat for Humanity ReStore"": fZQ3QjGMRELHG9f0WnN8vw","""The Home Depot"": 6vsjWxIMHs-34L9wuTBJRw","""The New Sweet Indulgence"": nplkC6vnh4qT9xH-vhup6w","""The Pita Pit"": DS3-yphtWDHAdXLtIoqdAA","""The Red Herring Vegetarian Restaurant"": kKCwp86xU9XKRnAALQDhrw","""The Ribeye"": PBmfdx-tC2D54FI3HtcKww","""The Y Eatery"": aKiE0aZ6vGyOlH-uVOHONw","""Thomas M. Siebel Center for Computer Science"": jDlvFXuxis4rC2NcWQbqig","""Timpone's"": 0grgvnq4GgoY-estWytUhg","""Tuesday Morning"": RDHH5aVSsblGfOf17P2Tjg","""UPS"": LQRcuOgluaRRMyTH4i3FPw","""University Group"": Hsvqv6AQQ13SvO0uGFu1zg","""University High School"": AqzK7Dr-9zMoxBpsZNlkoA","""University of Illinois Veterinary Teaching Hospital"": D5quwbQTguD4hqMAEARnBg","""University of Illinois"": ZiLOXVloAOtr9_cpQTpr8g","""Urbana Dog Park"": FM6hRJtjDwNeZWa64wpdRg","""Urbana Free Library"": wmhFEY8IctqfSBO0NDcb1w","""Wal-Mart"": 4VYi3I-nVttZeOmrGEXHjQ","""Walnut Street Tea"": pLBF8QrRkMicJcsjAqW7ag","""Ward & Associates Realtors"": yL0PKzDCONu3jHpJG4tQSQ","""Wienerschnitzel"": QHbrfA9nSNLFcevsfQk6dQ","""Will Am-Fm Radio"": YBNS9AHuSSMSVPGlsgvqbA","""William M Staerkel Planetarium"": DNBKTu7qoN_18JIybyIVPw","""Wingstop"": 4ZAy5ZCTWbduzzhtR0egHQ","""Women's Health Practice"": c_pScQ7eKYTesWkcqDMn-g","""Wonderdogs"": SuQpsHxcxCAB8kxPEIXiBg","""Woori Jib Restaurant"": GWS5s8HSm1zjqMLJD7wQLg","""World Harvest Foods"": LElQBiDw8HyU_LrbjP8C1A","""Yellow Checker Cab"": J6TGHzwKKq5bqBR4_WsnUA","""Yellowfin Japanese Restaurant"": C31ExBTn_6UxbTVkWPtNkg","""Yuko Hair Salon"": 4LXNVWV_Yp8HbkikmRQ2gg","""Za's Italian Cafe"": FUr2uEolARu7rv2TQYKPqg","""Zelma's"": h2vFVV5pJjn8arA7GR6mQg","""Zorba's Restaurant"": kyXEnWKQGWSThY6EcjORuw"
0,0.589713,0.189171,0.282108,0.008324,0.924193,6.457445e-06,0.934173,0.765077,0.151248,0.008078,1.7

1. Reading a table of users by how many years they've been active in
2. Reading the review file: this is used to find what the test users have rated

In [0]:
filepath = '/content/gdrive/My Drive/Summer Research 2019/Individual Work Folder/Jonathan/user_years_with_rating.csv'
df_y = pd.read_csv(filepath, index_col=0)

filepath = '/content/gdrive/My Drive/Summer Research 2019/Individual Work Folder/Ian/yelp_detailed_small.csv'
df_r = pd.read_csv(filepath)

df_y.head(10)

,user_name_id,years_with_rating
0,T. A.: ZT70ZzpVrWl013A-sId3Kw,9
1,Lisa: qmjoMFMZdLH69_6eGTGDZw,9
2,Jamie: D7vqQ2D1mmj_1EV1lNXsnQ,8
3,Mariam: VQKHUfAhPJ5wmNthq62mfQ,8
4,J: BsK9Oy3pTJ0SCdaKsgx_AQ,8
5,Edward: 5QOtcHU1SoqEqBCRR6FhsA,8
6,Heather: ymi0YEAIyZm9DN0H-OBb5A,8
7,Grace: Jiw1kr7W1IbbqL1p_V7d1A,8
8,Tony: StYuEMDMWO1IOdMz5zgE8Q,8
9,Kelly: 1TVhmXcDVqFTtyJNKEFVxw,8


## Getting the user test set
Here we extract the top x users as canidates for testing

In [0]:
# from the top users, find their reviews and make a dictionary <User, {set of users' businesses they reviewed}>
# x is assigned for the amount of users we want to search through
x = 100

df_y = df_y.iloc[0:x]
df_y = df_y['user_name_id'].tolist()

df_y[0:5]

['T. A.: ZT70ZzpVrWl013A-sId3Kw',
 'Lisa: qmjoMFMZdLH69_6eGTGDZw',
 'Jamie: D7vqQ2D1mmj_1EV1lNXsnQ',
 'Mariam: VQKHUfAhPJ5wmNthq62mfQ',
 'J: BsK9Oy3pTJ0SCdaKsgx_AQ']

In [0]:
dict_ub = {}
for i in df_y:
  df_temp = df_r[df_r['user_name_id'] == i]
  s = set(df_temp['business_name_id'])
  dict_ub[i] = s
len(dict_ub[df_y[0]])

54

## Comparing user review
We take the set from each user and intersect it with the total business set, and map it as the new set within the dict

In [0]:
s = set(df_v.columns)
for i in dict_ub.keys():
  dict_ub[i] = s.intersection(dict_ub[i])
len(dict_ub[df_y[0]])

17

## Generating predictions w/ cross-validation
For each user, we iterate through their set by:

1.   Multiply U and V
2.   Selecting one element out of the set to exclude
3.   Running Pearson correlation against the clusters with remaining set to find the most common cluster
4.   See what rating is predicted

In [0]:
# multiplying U and V
# converting to numpy to multiply
u = df_u.to_numpy()
v = df_v.to_numpy()
uv = np.matmul(u, v)
# converting back to dataframe
uv = pd.DataFrame(uv, columns = df_v.columns)

# list of error 
i_abs_diff = []

# For every <User, Set> pair
for i, j in dict_ub.items():
  # we exclude testing on any persons with less than 3 common businesses
  if len(j) < 3:
    continue
  # make a pivot table to extract the user's reviews
  temp_pivot = df_r[df_r['user_name_id'] == i]
  temp_pivot = temp_pivot.pivot(index = 'user_name_id', columns = 'business_name_id', values = 'stars')

  # For each item k in set j
  for k in j:
    temp_s = j.copy()

    # we test as if item k is not part of the set
    temp_s.remove(k)
    temp_s = list(temp_s)
    closest_cluster = (-1, -1)
    # for each row (cluster) in the UV matrix 
    for m in range(len(uv)):
      # make a list for both the UV and the user's reviews
      uv_list = [uv.loc[m][n] for n in temp_s]
      user_list = [temp_pivot.loc[i][n] for n in temp_s]
      pearsons = scipy.stats.pearsonr(uv_list, user_list)
      if (pearsons[1] > closest_cluster[1]):
        closest_cluster = (m, pearsons[1])
    # once the closest cluster has been found, we perform the tests
    if(closest_cluster[0] != -1):
      i_abs_diff.append(abs(uv.loc[closest_cluster[0]][k] - temp_pivot.loc[i][k]))
# print out MAE for entire set
print(sum(i_abs_diff) / len(i_abs_diff))

/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:3399: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


1.8392121121106115


### Improving prediction generation
Based on Dr. Wang's feedback, we can try these changes and see if there are any improvements:


*   Use cosine similarity instead of pearson's
*   Considering top-*k* clusters for prediction instead of just the single nearest cluster
*   For optimization, only multiply U and V by the user's rated businesses, instead of the full matrix which is not necessary for a particular user.



In [0]:
# k = number of clusters to consider
k = 3

error = []
# For every <User, Set> pair
for i, j in dict_ub.items():
  # v is the subset of (latent factors x Businesses) where the businesses 
  # are both in the volunteer set and i's (the current test user's) review set.
  # Everytime we want to multiply by only the users businesses, we need to remake the v matrix
  v = df_v[[x for x in j]]
  v_columns = v.columns
  
  # convert U and V to numpy arrays
  v = v.to_numpy()
  u = df_u.to_numpy()

  # matmul and convert back to DataFrame  
  uv = np.matmul(u, v)
  uv = pd.DataFrame(uv, columns = v_columns)

  # we reduce the df_r (review set) down to i's reviews and sort by date
  r = df_r[df_r['user_name_id'] == i]
  r = r.sort_values('date')
  r = r[r.business_name_id.isin(j)]

  # We run tests from 2 to n - 1 user reviews (a user requires a minimum of 3 reviews). Each test will: run cosine similarity, find the top 3 clusters, and compare the user's review for x business.
  
  for x in range(2, len(r)):
    # Select the business to test
    currb = r.iloc[x, :]['business_name_id']

    # Generate the list of businesses to run cosine similarity
    testset = r.iloc[0:x, :]['business_name_id'].tolist()
    l1 = [r[r['business_name_id'] == business].iloc[0]['stars'] for business in testset]

    # For each cluster, we take the businesses currently being tested and run cosine similarity
    csd = []
    for cluster in range(len(uv)):
      l2 = [uv.iloc[cluster][business] for business in testset]
      csdistance = scipy.spatial.distance.cosine(l1, l2)
      csd.append((csdistance, cluster))
    csd.sort(reverse = True)
    csd = csd[0:k]
    # With the top-k clusters in csd, we generate a prediction on currb, multiplying the score from the cluster by its weight
    weights = [cluster[0] for cluster in csd]
    sw = sum(weights)
    weights = map(lambda weight: weight / sw, weights)
    prediction = 0
    for weight, cluster in zip(weights, csd):
      prediction += (uv.iloc[cluster[1]][currb] * weight)
    error.append(abs(r.iloc[x, :]['stars'] - prediction))
sum(error) / len(error)

1.4307514088615507